In [ ]:
from PIL import Image # to read images
import os
import json

Image.LOAD_TRUNCATED_IMAGES = True

gt_jf="./scripts/instances_val.json"
for ii in range (4):
    # images_directory=f"./new_editted_classified_resized/{sub_dir}/{batch}/{ii+1}/"
    images_directory=f"./sea_state_classified/{ii+1}"
    for filename in os.listdir(images_directory):
        ori_filename = filename.split("_")[2]
        seed = ori_filename[0]
        im_id = ori_filename.split('.')[0]
        bbox=[]
        with open(gt_jf, encoding="utf-8") as f:
            data = json.load(f)
            count = 0
            for annotation in data["annotations"]:
                if int(annotation["image_id"]) == int(im_id) and int(annotation["category_id"]) ==2:
                    bbox = annotation["bbox"]
                    category = annotation["category_id"]
                    bbox = [bbox[0], bbox[1] ,bbox[0]+bbox[2], bbox[1]+bbox[3]]
                    try:
                        img=Image.open(f"{images_directory}/{filename}")
                        img=img.crop(bbox)
                        img.save(f"./crops/{ii+1}/{filename}__{bbox[0]}_{bbox[1]}_{bbox[2]}_{bbox[3]}.jpg")
                        
                    except IOError:
                        
                        pass
                    count += 1

In [ ]:
import os
import numpy as np
import cv2
import shutil

import torch

from monai.networks.nets import DenseNet121
from monai.transforms import *
from monai.data import Dataset, DataLoader

##Define MONAI transforms, Dataset and Dataloader to pre-process data
class SumDimension(Transform):
    def __init__(self, dim=1):
        self.dim = dim

    def __call__(self, inputs):
        return inputs.sum(self.dim)
class MyResize(Transform):
    def __init__(self, size=(120,120)):
        self.size = size
    def __call__(self, inputs):
        image=cv2.resize(inputs,dsize=(self.size[1],self.size[0]),interpolation=cv2.INTER_CUBIC)
        image2=image[30:90,30:90]
        return image2
class Astype(Transform):
    def __init__(self, type='uint8'):
        self.type = type
    def __call__(self, inputs):
        return inputs.astype(self.type)

val_transforms = Compose([
    LoadImage(image_only=True),
    Resize((-1,1)),
    Astype(),
    SumDimension(2),
    Astype(),
    MyResize(),
    AddChannel(),    
    ToTensor(),
])

act = Activations(softmax=True)
to_onehot = AsDiscrete(to_onehot=6, n_classes=6)
##to_onehot=True/False` is deprecated, please use `to_onehot=num_classes` instead.
class MedNISTDataset(Dataset):

    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]



device = torch.device("cuda:0")   #"cuda:0"
model = DenseNet121(
    spatial_dims=2,            
    in_channels=1,
    out_channels=2,
).to(device)

model.load_state_dict(torch.load('./models/boat_model.pth'))
model.eval()

for ii in range(4):
    main_dir = f"./crops/{ii+1}"
    for filename in os.listdir(main_dir):
        # if (int(filename.split('_')[0])<100):
        shutil.move(f"{main_dir}/{filename}", f"./temp_boat/1/{filename}")
        y_true = list()
        y_pred = list()

        editted_test_dir='./temp_boat'
        #test
        t_class_names0 = os.listdir(editted_test_dir)
        t_class_names = sorted(t_class_names0)

        t_num_class = len(t_class_names)
        t_image_files = [[os.path.join(editted_test_dir, t_class_name, x) 
                    for x in os.listdir(os.path.join(editted_test_dir, t_class_name))] 
                    for t_class_name in t_class_names]

        t_image_file_list = []
        t_image_label_list = []
        for i, class_name in enumerate(t_class_names):
            t_image_file_list.extend(t_image_files[i])
            t_image_label_list.extend([i] * len(t_image_files[i]))

        testX=np.array(t_image_file_list)
        testY=np.array(t_image_label_list)

        editted_test_ds = MedNISTDataset(testX, testY, val_transforms)
        editted_test_loader = DataLoader(editted_test_ds, batch_size=32, num_workers=2)

        with torch.no_grad():
            for test_data in editted_test_loader:
                test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
                pred = model(test_images.float()).argmax(dim=1)
                for i in range(len(pred)):
                    y_pred.append(pred[i].item())
                    shutil.move(f"./temp_boat/1/{filename}", f"./crops_checked/{ii+1}/{pred[0].item()}/{filename}")

In [ ]:
for ii in range(4):
    image_directory = f"./sea_state_classified/{ii+1}"
    crop_directory = f"./crops_checked/{ii+1}/1"
    image_list = []
    for filename in os.listdir(crop_directory):
        if filename.split('__')[0] not in image_list:
            image_list.append(filename.split('__')[0])
    for filename in os.listdir(image_directory):
        if filename.split('.')[0] not in image_list:
            os.remove(f"{image_directory}/{filename}")
